In [64]:
__SCRIPT_VERSION__ = "3.0"
__SCRIPT_DATE__ = "18/May/2023"

In [65]:
from fastlane_bot.bot import CarbonBot as Bot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, Pair
from fastlane_bot.tools.analyzer import CPCAnalyzer
from fastlane_bot.tools.cryptocompare import CryptoCompare
import requests
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCAnalyzer))
import pandas as pd
import datetime
import time
import json
from hashlib import md5
from fastlane_bot import __VERSION__

CarbonBot v3-b2.1 (03/May/2023)
ConstantProductCurve v2.13 (15/May/2023)
CPCAnalyzer v1.5 (18/May/2023)


# Mainnet Arbitrage Monitoring Bot [A015]

In [66]:
cid = lambda pair: md5(pair.encode()).hexdigest()
cid("WETH-6Cc2/USDC-eB48")

'ead90114986e463b0157c49422d8d465'

In [67]:
bot     = Bot()
CCm     = bot.get_curves()
fn = f"../data/A014-{int(time.time())}.csv.gz"
print (f"Saving as {fn}")
CCm.asdf().to_csv(fn, compression = "gzip")

Saving as ../data/A014-1684432582.csv.gz


In [68]:
class TokenAddress():
    def __init__(self, db):
        self._db = db
        
    def addr_from_ticker(self, ticker):
        return self._db.get_token(key=ticker).address
    a = addr_from_ticker
    
    def ticker_from_addr(self, addr):
        raise NotImplemented()
TA = TokenAddress(bot.db)      
TA.a("WETH-6Cc2")

'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

## Header and metadata

In [69]:
now = datetime.datetime.now()
print("*"*100)
print(f"ARBITRAGE ANALYSIS RUN @ {now.isoformat().split('.')[0]}Z [{int(now.timestamp())}]")
print("*"*100)

****************************************************************************************************
ARBITRAGE ANALYSIS RUN @ 2023-05-18T18:56:22Z [1684432582]
****************************************************************************************************


## Read curves

### Read Carbon curves

In [70]:
#CCm     = CPCContainer.from_df(pd.read_csv("../data/A014-1684148163.csv.gz"))
# CCu3    = CCm.byparams(exchange="uniswap_v3")
# CCu2    = CCm.byparams(exchange="uniswap_v2")
# CCs2    = CCm.byparams(exchange="sushiswap_v2")
CCc1      = CCm.byparams(exchange="carbon_v1")              # all Carbon positions
CCnc1     = CCm.byparams(exchange="carbon_v1", _inv=True)   # all non-Carbon positions
# tc_u3   = CCu3.token_count(asdict=True)
# tc_u2   = CCu2.token_count(asdict=True)
# tc_s2   = CCs2.token_count(asdict=True)
# tc_c1   = CCc1.token_count(asdict=True)
# CAm     = CPCAnalyzer(CCm)
CAc1 = CPCAnalyzer(CCc1)
pairs  = CAc1.pairsc()

In [71]:
help(CCm.byparams)

Help on method byparams in module fastlane_bot.tools.cpc:

byparams(*, _asgenerator=None, _ascc=None, _inv=False, **params) method of fastlane_bot.tools.cpc.CPCContainer instance
    returns all curves by params (as tuple, generator or CC object)
    
    :_inv:      if True, returns all curves that do NOT match the params
    :params:    keyword arguments in the form param=value
    :returns:   tuple, generator or container object (default)



now = datetime.datetime.now()
int(now.timestamp()), now.isoformat()## Print heading

### Read prices and create proxy curves

#### Preparations

In [72]:
tokens0 = CAc1.tokens()
tokens0

{'0x0-1AD5',
 'ARB-4ad1',
 'BNT-FF1C',
 'CRV-cd52',
 'DAI-1d0F',
 'DEXT-C75a',
 'ETH2x_FLI-65BD',
 'LBR-aCcA',
 'LINK-86CA',
 'LYXe-be6D',
 'MATIC-eBB0',
 'PEPE-1933',
 'RPL-A51f',
 'SMT-7173',
 'TSUKA-69eD',
 'USDC-eB48',
 'USDT-1ec7',
 'WBTC-C599',
 'WETH-6Cc2',
 'XCHF-fc08',
 'eRSDL-D3A6',
 'stETH-fE84',
 'vBNT-7f94'}

In [73]:
print("\n\n"+"="*100)
print("REMOVED TOKENS")
print("="*100)



REMOVED TOKENS


In [74]:
REMOVED_TOKENS = {"0x0-1AD5", "LBR-aCcA"}
print(REMOVED_TOKENS)

{'0x0-1AD5', 'LBR-aCcA'}


In [75]:
tokens = tokens0 - REMOVED_TOKENS
pairs  = CAc1.CC.filter_pairs(bothin=tokens)
tokens_addr = {tkn: TA.a(tkn) for tkn in tokens}
tokens_addrr = {v.lower():k for k,v in tokens_addr.items()}
print("\n\n"+"="*100)
print("TOKEN ADDRESSES")
print("="*100)
for k,v in tokens_addr.items():
    print(f"{k:20} {v}")
tokens_addr, tokens_addrr



TOKEN ADDRESSES
ARB-4ad1             0xB50721BCf8d664c30412Cfbc6cf7a15145234ad1
BNT-FF1C             0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C
stETH-fE84           0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84
PEPE-1933            0x6982508145454Ce325dDbE47a25d4ec3d2311933
WETH-6Cc2            0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
XCHF-fc08            0xB4272071eCAdd69d933AdcD19cA99fe80664fc08
MATIC-eBB0           0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0
WBTC-C599            0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599
CRV-cd52             0xD533a949740bb3306d119CC777fa900bA034cd52
LINK-86CA            0x514910771AF9Ca656af840dff83E8264EcF986CA
ETH2x_FLI-65BD       0xAa6E8127831c9DE45ae56bB1b0d4D4Da6e5665BD
TSUKA-69eD           0xc5fB36dd2fb59d3B98dEfF88425a3F425Ee469eD
eRSDL-D3A6           0x5218E472cFCFE0b64A064F055B43b4cdC9EfD3A6
USDC-eB48            0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
LYXe-be6D            0xA8b919680258d369114910511cc87595aec0be6D
USDT-1ec7            0

({'ARB-4ad1': '0xB50721BCf8d664c30412Cfbc6cf7a15145234ad1',
  'BNT-FF1C': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C',
  'stETH-fE84': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84',
  'PEPE-1933': '0x6982508145454Ce325dDbE47a25d4ec3d2311933',
  'WETH-6Cc2': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
  'XCHF-fc08': '0xB4272071eCAdd69d933AdcD19cA99fe80664fc08',
  'MATIC-eBB0': '0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0',
  'WBTC-C599': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
  'CRV-cd52': '0xD533a949740bb3306d119CC777fa900bA034cd52',
  'LINK-86CA': '0x514910771AF9Ca656af840dff83E8264EcF986CA',
  'ETH2x_FLI-65BD': '0xAa6E8127831c9DE45ae56bB1b0d4D4Da6e5665BD',
  'TSUKA-69eD': '0xc5fB36dd2fb59d3B98dEfF88425a3F425Ee469eD',
  'eRSDL-D3A6': '0x5218E472cFCFE0b64A064F055B43b4cdC9EfD3A6',
  'USDC-eB48': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  'LYXe-be6D': '0xA8b919680258d369114910511cc87595aec0be6D',
  'USDT-1ec7': '0xdAC17F958D2ee523a2206206994597C13D831ec7',
  'DAI-1d0F': '0x6

#### CryptoCompare

In [76]:
tokens_cc = [Pair.n(x) for x in tokens]
tokens_cc

['ARB',
 'BNT',
 'stETH',
 'PEPE',
 'WETH',
 'XCHF',
 'MATIC',
 'WBTC',
 'CRV',
 'LINK',
 'ETH2x_FLI',
 'TSUKA',
 'eRSDL',
 'USDC',
 'LYXe',
 'USDT',
 'DAI',
 'vBNT',
 'SMT',
 'DEXT',
 'RPL']

In [77]:
token_prices_usd_cc = CryptoCompare(apikey=True, verbose=False).query_tokens(tokens_cc)
token_prices_usd_cc

{'ARB': 1.141,
 'BNT': 0.4153,
 'STETH': 1775.07,
 'PEPE': 1.52e-06,
 'WETH': 1793.64,
 'XCHF': 1.005,
 'MATIC': 0.8458,
 'WBTC': 26535.14,
 'CRV': 0.8017,
 'LINK': 6.497,
 'TSUKA': 0.05181,
 'ERSDL': 0.002422,
 'USDC': 1.0,
 'LYXE': 12.96,
 'USDT': 1.0,
 'DAI': 1.0,
 'SMT': 0.001012,
 'DEXT': 0.6141,
 'RPL': 48.79}

In [78]:
missing_cc = set(tokens_cc) - set(token_prices_usd_cc)
missing_cc

{'ETH2x_FLI', 'LYXe', 'eRSDL', 'stETH', 'vBNT'}

In [79]:
token_prices_usd = token_prices_usd_cc
P0 = lambda tknb,tknq: token_prices_usd[tknb.upper()]/token_prices_usd[tknq.upper()]
def P(pair):
    try: 
        return P0(*Pair.n(pair).split("/"))
    except KeyError:
        return None

prices_by_pair = {pair: P(pair) for pair in pairs}
prices_n_by_pair = {Pair.n(pair): p for pair, p in prices_by_pair.items()}
print("\n\n"+"="*100)
print("PRICES BY PAIR (CRYPTOCOMPARE)")
print("="*100)
for k,v in prices_n_by_pair.items():
    if not v is None:
        print(f"{k:20} {v:20,.6f}")
    else:
        print(f"{k:20}                  ---")



PRICES BY PAIR (CRYPTOCOMPARE)
WETH/USDT                    1,793.640000
WBTC/WETH                       14.794017
WBTC/USDC                   26,535.140000
USDT/USDC                        1.000000
ETH2x_FLI/WETH                        ---
SMT/WETH                         0.000001
DEXT/USDC                        0.614100
DAI/USDC                         1.000000
vBNT/USDC                             ---
WETH/BNT                     4,318.901999
ARB/MATIC                        1.349019
LINK/USDC                        6.497000
LINK/USDT                        6.497000
BNT/USDC                         0.415300
WBTC/USDT                   26,535.140000
WETH/DAI                     1,793.640000
CRV/USDC                         0.801700
DAI/USDT                         1.000000
RPL/XCHF                        48.547264
vBNT/BNT                              ---
WETH/USDC                    1,793.640000
PEPE/WETH                        0.000000
LYXe/USDC                       12.960000
e

In [80]:
proxy_curves_cc = [
    CPC.from_pk(p=price, pair=pair, k=1000, cid=cid(pair+"CG"), params=dict(exchange="ccomp")) 
    for pair, price in prices_by_pair.items() if not price is None
]
#proxy_curves_cc

#### CoinGecko

In [81]:
addr_s = ",".join(x for x in tokens_addr.values())
url = "https://api.coingecko.com/api/v3/simple/token_price/ethereum"
params = dict(contract_addresses=addr_s, vs_currencies="usd")
r = requests.get(url, params=params)
token_prices_usd_cg_raw = {tokens_addrr[k]: v["usd"] for k,v in r.json().items()}
token_prices_usd_cg = {Pair.n(tokens_addrr[k]).upper(): v["usd"] for k,v in r.json().items()}
token_prices_usd_cg_raw

{'MATIC-eBB0': 0.850905,
 'DAI-1d0F': 0.998509,
 'TSUKA-69eD': 0.051898,
 'DEXT-C75a': 0.591853,
 'stETH-fE84': 1791.79,
 'LINK-86CA': 6.56,
 'LYXe-be6D': 13.12,
 'ARB-4ad1': 1.15,
 'SMT-7173': 0.075252,
 'eRSDL-D3A6': 0.00243755,
 'CRV-cd52': 0.809008,
 'USDT-1ec7': 0.997863,
 'PEPE-1933': 1.55e-06,
 'RPL-A51f': 48.8,
 'WETH-6Cc2': 1790.48,
 'XCHF-fc08': 1.11,
 'BNT-FF1C': 0.419064,
 'vBNT-7f94': 0.335932,
 'ETH2x_FLI-65BD': 11.72,
 'WBTC-C599': 26723,
 'USDC-eB48': 0.993973}

In [82]:
missing_cg = set(tokens_addr) - set(token_prices_usd_cg_raw)
missing_cg

set()

In [83]:
token_prices_usd = token_prices_usd_cg
P0 = lambda tknb,tknq: token_prices_usd[tknb.upper()]/token_prices_usd[tknq.upper()]
def P(pair):
    try: 
        return P0(*Pair.n(pair).split("/"))
    except KeyError:
        return None

prices_by_pair = {pair: P(pair) for pair in pairs}
prices_n_by_pair = {Pair.n(pair): p for pair, p in prices_by_pair.items()}
print("\n\n"+"="*100)
print("PRICES BY PAIR (COINGECKO)")
print("="*100)
for k,v in prices_n_by_pair.items():
    if not v is None:
        print(f"{k:20} {v:20,.6f}")
    else:
        print(f"{k:20}                  ---")



PRICES BY PAIR (COINGECKO)
WETH/USDT                    1,794.314450
WBTC/WETH                       14.925048
WBTC/USDC                   26,885.036113
USDT/USDC                        1.003914
ETH2x_FLI/WETH                   0.006546
SMT/WETH                         0.000042
DEXT/USDC                        0.595442
DAI/USDC                         1.004564
vBNT/USDC                        0.337969
WETH/BNT                     4,272.569345
ARB/MATIC                        1.351502
LINK/USDC                        6.599777
LINK/USDT                        6.574049
BNT/USDC                         0.421605
WBTC/USDT                   26,780.229350
WETH/DAI                     1,793.153592
CRV/USDC                         0.813913
DAI/USDT                         1.000647
RPL/XCHF                        43.963964
vBNT/BNT                         0.801625
WETH/USDC                    1,801.336656
PEPE/WETH                        0.000000
LYXe/USDC                       13.199554
eRSDL

In [84]:
proxy_curves_cg = [
    CPC.from_pk(p=price, pair=pair, k=1000, cid=cid(pair+"CG"), params=dict(exchange="cgecko")) 
    for pair, price in prices_by_pair.items() if not price is None
]
#proxy_curves_cg

#### Assembly

In [85]:
# CCother = CCu3.bypairs(CCc1.pairs())
CCcg = CPCContainer(proxy_curves_cg)
CCcc = CPCContainer(proxy_curves_cc)
CCfull = CCc1.copy().add(CCcg).add(CCcc)
#CAother = CPCAnalyzer(CCother)
CAfull = CPCAnalyzer(CCfull)
CAnc1  = CPCAnalyzer(CCnc1)
print(f"CAfull: {len(CAfull.CC):4} entries")
print(f"CAnc1:  {len(CAnc1.CC):4} entries")

CAfull:  129 entries
CAnc1:    29 entries


## By-pair data for Carbon

### Count by pairs

In [86]:
df = CAc1.count_by_pairs()
print("\n\n"+"="*100)
print("PAIRS")
print("="*100)
print(df)
#df



PAIRS
                          count
pair                           
WETH-6Cc2/USDC-eB48          17
vBNT-7f94/BNT-FF1C           10
WETH-6Cc2/BNT-FF1C            9
USDT-1ec7/USDC-eB48           5
stETH-fE84/WETH-6Cc2          4
WBTC-C599/WETH-6Cc2           3
DAI-1d0F/USDC-eB48            3
ARB-4ad1/MATIC-eBB0           2
DAI-1d0F/USDT-1ec7            2
WBTC-C599/USDC-eB48           2
PEPE-1933/WETH-6Cc2           2
CRV-cd52/USDC-eB48            2
LINK-86CA/USDT-1ec7           2
WETH-6Cc2/USDT-1ec7           2
0x0-1AD5/WETH-6Cc2            2
BNT-FF1C/USDC-eB48            1
DEXT-C75a/USDC-eB48           1
LBR-aCcA/WETH-6Cc2            1
RPL-A51f/XCHF-fc08            1
WBTC-C599/USDT-1ec7           1
TSUKA-69eD/USDC-eB48          1
LYXe-be6D/USDC-eB48           1
LINK-86CA/USDC-eB48           1
WETH-6Cc2/DAI-1d0F            1
ETH2x_FLI-65BD/WETH-6Cc2      1
SMT-7173/WETH-6Cc2            1
eRSDL-D3A6/WETH-6Cc2          1
vBNT-7f94/USDC-eB48           1


In [87]:
print("\n\n                    CARBON     CGECKO   CCOMP")
print(f"Pairs:                {len(pairs):4}    {len(CCcg.pairs()):7} {len(CCcc.pairs()):7}")
print(f"Tokens:               {len(tokens):4}    {len(CCcg.tokens()):7} {len(CCcc.tokens()):7}")
print(f"Curves:               {len(CAc1.CC):4}    {len(CCcg):7} {len(CCcc):7}")



                    CARBON     CGECKO   CCOMP
Pairs:                  26         26      23
Tokens:                 21         21      19
Curves:                 80         26      23


### Calculate by-pair statistics

In [88]:
pasdf    = CAfull.pool_arbitrage_statistics()
pasnc1df = CAnc1.pool_arbitrage_statistics(only_pairs_with_carbon=False)

### Print by-pair statistics

In [89]:
def prints(*x):
    global s
    s += " ".join([str(x_) for x_ in x])
    s += "\n"
out_by_pair = dict()
carbon_by_pair = dict()
other_by_pair = dict()

for pair in list(pairs):
    s = ""
    prints("\n\n"+"="*100)
    prints(f"Pair = {pair}")
    prints("="*100)
    df = pasdf.loc[Pair.n(pair)]
    try:
        nc1df = pasnc1df.loc[Pair.n(pair)]
    except:
        nc1df = pd.DataFrame()
    hasproxydata = len(df.reset_index()[df.reset_index()["exchange"]=="cgecko"])>0
    if hasproxydata:
        prints("\n--- ALL CARBON AND REFERENCE POSITIONS ---")
        prints(df.to_string())
        carbon_by_pair[pair] = [[k,v] for k,v in df.to_dict(orient="index").items()]
        prints("\n--- IN-THE-MONEY POSITIONS ---")
        dfitm = df[df["itm"]=="x"]
        if len(dfitm) > 0:
            prints(dfitm.to_string())
        else:
            prints("-None-")
        prints("\n--- ALL NON-CARBON POSITIONS ---")
        if len(nc1df) > 0:
            prints(nc1df.to_string())
        else:
            prints("-None-")
        other_by_pair[pair] = [[k,v] for k,v in nc1df.to_dict(orient="index").items()]
        
    else:
        prints("\n--- NO PRICE DATA AVAILABLE ---")
    
    out_by_pair[pair] = s
    print(s)




Pair = WETH-6Cc2/USDT-1ec7

--- ALL CARBON AND REFERENCE POSITIONS ---
                          price        vl itm  b  s                                    bsv
exchange  cid0                                                                            
carbon_v1 691723-0  1600.000160  0.003125      b          buy-WETH @ 1600.00 USDT per WETH
ccomp     e60f9a1d  1793.640000  1.493353   x  b  s  buy-sell-WETH @ 1793.64 USDT per WETH
cgecko    e60f9a1d  1794.314450  1.493072   x  b  s  buy-sell-WETH @ 1794.31 USDT per WETH
carbon_v1 691656-0  1891.000189  0.002644   x  b          buy-WETH @ 1891.00 USDT per WETH

--- IN-THE-MONEY POSITIONS ---
                          price        vl itm  b  s                                    bsv
exchange  cid0                                                                            
ccomp     e60f9a1d  1793.640000  1.493353   x  b  s  buy-sell-WETH @ 1793.64 USDT per WETH
cgecko    e60f9a1d  1794.314450  1.493072   x  b  s  buy-sell-WETH @ 1794.31

## Summary data

### Create summary data

In [90]:
itmcarbdf = pasdf.query("exchange == 'carbon_v1'").query("itm == 'x'")

In [91]:
itmcarb_pairs = sorted({x[0] for x in tuple(itmcarbdf.index)})
itmcarb_pairs

['ARB/MATIC',
 'CRV/USDC',
 'LINK/USDT',
 'WBTC/USDC',
 'WETH/BNT',
 'WETH/USDC',
 'WETH/USDT',
 'vBNT/BNT']

In [92]:
itmcarb_pos = itmcarbdf.reset_index().to_dict(orient="records")
itmcarb_pos[:2]

[{'pair': 'ARB/MATIC',
  'exchange': 'carbon_v1',
  'cid0': '806240-1',
  'price': 1.4285714285714268,
  'vl': 141.80602335295742,
  'itm': 'x',
  'b': 'b',
  's': '',
  'bsv': 'buy-ARB @ 1.43 MATIC per ARB'},
 {'pair': 'CRV/USDC',
  'exchange': 'carbon_v1',
  'cid0': '612490-0',
  'price': 0.89999990851846,
  'vl': 1.7856990703983344,
  'itm': 'x',
  'b': 'b',
  's': '',
  'bsv': 'buy-CRV @ 0.90 USDC per CRV'}]

In [93]:
itmcarb_pos_bypair = {
    pair: [x for x in itmcarb_pos if x["pair"] == pair]
    for pair in itmcarb_pairs
}
#itmcarb_pos_bypair

In [94]:
missing_pairs = [pair for pair, price in prices_by_pair.items() if price is None]
missing_pairs

[]

In [95]:
carbon_by_pair

{'WETH-6Cc2/USDT-1ec7': [[('carbon_v1', '691723-0'),
   {'price': 1600.000159878855,
    'vl': 0.0031249996877366426,
    'itm': '',
    'b': 'b',
    's': '',
    'bsv': 'buy-WETH @ 1600.00 USDT per WETH'}],
  [('ccomp', 'e60f9a1d'),
   {'price': 1793.6400000000006,
    'vl': 1.4933525758030302,
    'itm': 'x',
    'b': 'b',
    's': 's',
    'bsv': 'buy-sell-WETH @ 1793.64 USDT per WETH'}],
  [('cgecko', 'e60f9a1d'),
   {'price': 1794.3144499796063,
    'vl': 1.493071887465954,
    'itm': 'x',
    'b': 'b',
    's': 's',
    'bsv': 'buy-sell-WETH @ 1794.31 USDT per WETH'}],
  [('carbon_v1', '691656-0'),
   {'price': 1891.0001887578283,
    'vl': 0.002644103384931141,
    'itm': 'x',
    'b': 'b',
    's': '',
    'bsv': 'buy-WETH @ 1891.00 USDT per WETH'}]],
 'WBTC-C599/WETH-6Cc2': [[('carbon_v1', '709362-1'),
   {'price': 14.28571428571426,
    'vl': 0.41708702154504046,
    'itm': '',
    'b': 'b',
    's': '',
    'bsv': 'buy-WBTC @ 14.29 WETH per WBTC'}],
  [('ccomp', 'c59d6071')

### Convert summary data to Telegram

In [96]:
telegram_data = dict(
    script_version = __SCRIPT_VERSION__,         # version number of the script producing this record
    script_version_dt = __SCRIPT_DATE__,         # ditto date
    time_ts = int(now.timestamp()),              # timestamp (epoch)
    time_iso = now.isoformat().split('.')[0],    # timestap (iso format)
    prices_usd = token_prices_usd,               # token prices (usd)
    pairs = list(pairs),                         # all pairs
    pairs_n = len(pairs),                        # ...number
    itm_pairs = itmcarb_pairs,                   # pairs that have curves in the money (list)
    itm_pairs_n = len(itmcarb_pairs),            # ...number
    itm_pos = itmcarb_pos,                       # carbon and reference positions that are in the money (list)
    itm_pos_n = len(itmcarb_pos),                # ...number
    all_pos_bp = carbon_by_pair,                 # all carbon and reference positions by pair (dict->list)
    all_pos_bp_n = len(carbon_by_pair),          # ...number
    other_pos_bp = other_by_pair,                # all other positions (dict->list)
    other_pos_bp_n = len(other_by_pair),         # ...number
    itm_pos_bypair = itmcarb_pos_bypair,         # ditto, but dict[pair] -> list
    missing_pairs = missing_pairs,               # missing pairs
    missing_pairs_n = len(missing_pairs),        # ...number
    removed_tokens = list(REMOVED_TOKENS),       # removed tokens
    removed_tokens_n = len(REMOVED_TOKENS),      # ...number
    out_by_pair = out_by_pair                    # output by pair
)

In [97]:
td = telegram_data
s = ""
s += f"="*47
s += f"\nARBITRAGE RUN @ {td['time_iso']}Z\n"
s += f"="*47+"\n"
s += f"Removed tokens:          {td['removed_tokens_n']:3}\n"
s += f"Total pairs:             {td['pairs_n']:3}\n"
s += f"Missing pairs:           {td['missing_pairs_n']:3}\n"
s += f"In-the-money pairs:      {td['itm_pairs_n']:3}\n"
s += f"In-the-money curves:     {td['itm_pos_n']:3}\n"
total_vl_usd = 0
total_arbval = 0
s += "-----------------------------------------------\n"
s += "PAIR         CID          VLOCK    ARBPC    VAL\n"
s += "-----------------------------------------------\n"
for p in td['itm_pos']:
    price_pair = prices_n_by_pair[p['pair']] or 0
    price_pc = f"{abs(price_pair/p['price']-1)*100:8.1f}%"
    vl_token = p['pair'].split('/')[0].split("-")[0]
    vl_token_price = token_prices_usd.get(vl_token.upper())
    vl_usd = p['vl']*vl_token_price
    total_vl_usd += vl_usd
    arbval = vl_usd * abs(price_pair/p['price']-1)
    if price_pc.endswith("100.0%"): 
        price_pc = "         "
        arbval = 0
    total_arbval += arbval
    s += f"{p['pair']:12} "
    s += f"{p['cid0'][-8:]:8} "
    s += f"{vl_usd:9,.0f}"
    s += f"{price_pc} "
    s += f"{arbval:6,.0f}"
    #s += f"[{p['bsv']}; p={price_pair:,.2f}]"
    #s += f"\n{p}"
    s += "\n"
s += "-----------------------------------------------\n"
s += f"TOTAL {total_vl_usd:25,.0f}   {100*total_arbval/total_vl_usd:5.1f}% {total_arbval:6,.0F}\n"
s += "===============================================\n"
s += "\n\n"
telegram_data["summary_text"] = s
print()
print(s)


ARBITRAGE RUN @ 2023-05-18T18:56:22Z
Removed tokens:            2
Total pairs:              26
Missing pairs:             0
In-the-money pairs:        8
In-the-money curves:      10
-----------------------------------------------
PAIR         CID          VLOCK    ARBPC    VAL
-----------------------------------------------
ARB/MATIC    806240-1       163     5.4%      9
CRV/USDC     612490-0         1     9.6%      0
LINK/USDT    960408-0         0     4.7%      0
WBTC/USDC    537493-0       485     0.7%      3
WETH/BNT     326076-0         5     2.6%      0
WETH/USDC    057365-0        18     1.6%      0
WETH/USDT    691656-0         5     5.1%      0
vBNT/BNT     748977-1       168     0.2%      0
vBNT/BNT     749015-0    16,823     9.4%  1,584
vBNT/BNT     748990-0         0    10.9%      0
-----------------------------------------------
TOTAL                    17,669     9.0%  1,598





In [98]:
with open("Analysis_015.latest.out", "w") as f:
    f.write(s)

In [99]:
with open("Analysis_015.latest.json", "w") as f:
    f.write(json.dumps(telegram_data))

In [100]:
None or 0

0

## Review

In [101]:
#print(CCfull.bycids(endswith="612490-0")[0].description())